In [2]:
!pip install -qU  langchain-core==0.2.9
!pip install -qU langchain-community==0.2.5
!pip install -qU  crewai[tools]==0.32.0
!pip install -qU langchain-groq==0.1.5

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain 0.2.11 requires langchain-core<0.3.0,>=0.2.23, but you have langchain-core 0.2.9 which is incompatible.
langchain-text-splitters 0.2.2 requires langchain-core<0.3.0,>=0.2.10, but you have langchain-core 0.2.9 which is incompatible.


In [1]:
import pandas as pd

In [2]:
import json
import os
import sqlite3
from dataclasses import asdict, dataclass
from datetime import datetime, timezone
from pathlib import Path
from textwrap import dedent
from typing import Any, Dict, List, Tuple, Union

import pandas as pd
from crewai import Agent, Crew, Process, Task
from crewai_tools import tool
# from google.colab import userdata
from langchain.schema import AgentFinish
from langchain.schema.output import LLMResult
from langchain_community.tools.sql_database.tool import (
    InfoSQLDatabaseTool,
    ListSQLDatabaseTool,
    QuerySQLCheckerTool,
    QuerySQLDataBaseTool,
)
from langchain_community.utilities.sql_database import SQLDatabase
from langchain_core.callbacks.base import BaseCallbackHandler
from langchain_core.prompts import ChatPromptTemplate
from langchain_groq import ChatGroq


In [3]:
df = pd.read_excel(r"C:\Users\HP\Documents\POC_X\Milestones 1.xlsx")
df.head()

,Project Name,Milestone Name,Owner Name,Start Date,End Date,Milestone Status,Milestone ID,Owner ID,Project ID,Created Time,Duration,Actual Time Taken,Milestone Completion Mode,Milestone ID String,Project ID String
0,Airowire - Zoho Analytics,Milestone Two-Implementation of Zoho Analytics,Uday Desai,2024-01-09 09:30:00,2024-03-22 19:00:00,Upcoming,749813000020001024,43003637,749813000020003968,2024-01-09 15:40:00,74,70,On Time,749813000020001024,749813000020003968
1,NMDSC - Zoho Connect,Milestone Two-Implementation completion of Zoh...,Prableen Kaur,2024-03-11 00:00:00,2024-03-29 23:59:00,Upcoming,749813000020594048,792886644,749813000020593024,2024-03-06 10:20:00,19,8,On Time,749813000020594048,749813000020593024
2,Juspay - Zoho Desk & Zoho SalesIQ - PA,Milestone Three-Annual maintenance charges - P...,Uday Desai,2024-04-01 00:00:00,2025-03-31 23:59:00,Upcoming,749813000020686976,43003637,749813000020685952,2024-03-12 19:26:00,365,-13,On Time,749813000020686976,749813000020685952
3,CEEW - Zoho ONE - ERP Implementation,Milestone Two-Implementation of Zoho Creator -...,Vignesh R,2024-02-01 00:00:00,2024-03-15 00:00:00,Overdue,749813000020093056,704175602,749813000020089984,2024-01-16 14:48:00,44,47,Late,749813000020093056,749813000020089984
4,CEEW - Zoho ONE - ERP Implementation,Milestone Four-Implementation of Zoho Books - ...,Stuti S. Kulshrestha,2024-03-13 00:00:00,2024-04-09 23:59:00,Upcoming,749813000020093952,808004288,749813000020089984,2024-01-16 14:48:00,28,6,On Time,749813000020093952,749813000020089984


In [4]:
import sqlite3

In [5]:
connection = sqlite3.connect("milestones.db")
df.to_sql(name="milestones", con=connection)

78

In [39]:
db = SQLDatabase.from_uri("sqlite:///milestones.db")

In [49]:
query="""
SELECT [Milestone Name], [Duration] 
FROM milestones 
WHERE [Milestone Name] LIKE '%zoho crm%'"""

In [52]:
query1="""SELECT DISTINCT "Project Name" FROM milestones WHERE "Owner Name" LIKE '%Waheeba%'"""

In [59]:
@tool("execute_sql")
def execute_sql(sql_query: str) -> str:
    """Execute a SQL query against the database. Returns the result"""
    return QuerySQLDataBaseTool(db=db).invoke(sql_query)

execute_sql.run("""SELECT * FROM milestones WHERE Duration > Actual Time Taken;""")

Using Tool: execute_sql


'Error: (sqlite3.OperationalError) near "Time": syntax error\n[SQL: SELECT * FROM milestones WHERE Duration > Actual Time Taken;]\n(Background on this error at: https://sqlalche.me/e/20/e3q8)'